<a href="https://colab.research.google.com/github/teresaliau/DSA4213Assignement3/blob/main/Experiments%2Binteractve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

1. Download the models from READMe or at these links

Full Fine-tuned model: https://drive.google.com/drive/folders/1exOmAGt4iIYT3tBniyPrJHAzLbUnJLKy?usp=sharing

LORA: https://drive.google.com/drive/folders/1G0PzrUMzjJ4ZNlyt_h_4X5t6X8fY0BK-?usp=sharing

2. Mount your Google Drive
3. Load models : Input your own path to model!!!
4. Run the notebook
5. ineteractive section at the end of the notebook


In [1]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json


print("Loading tokenizer")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading tokenizer


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Input your own path to downloaded model


In [2]:
# Input your own path to the models in your google drive
# CHANGE THIS!!
print("Input your own path to model")
model_path_full = '/content/drive/MyDrive/DSA4213_Models/model_full_finetuned'
model_path_lora = '/content/drive/MyDrive/DSA4213_Models/model_lora_adapter'

# Load models
model_full = AutoModelForSequenceClassification.from_pretrained(model_path_full, num_labels=6)
model_lora = AutoModelForSequenceClassification.from_pretrained(model_path_lora, num_labels=6)

Input your own path to model


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Sample Texts to test Models

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import json



# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_full.to(device)
model_lora.to(device)

# Define label names
label_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


# Prediction function
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Full FT prediction
    model_full.eval()
    with torch.no_grad():
        outputs_full = model_full(**inputs)
        probs_full = torch.nn.functional.softmax(outputs_full.logits, dim=-1)[0]
        pred_full = torch.argmax(probs_full).item()

    # LoRA prediction
    model_lora.eval()
    with torch.no_grad():
        outputs_lora = model_lora(**inputs)
        probs_lora = torch.nn.functional.softmax(outputs_lora.logits, dim=-1)[0]
        pred_lora = torch.argmax(probs_lora).item()

    # Display results and compare predictions
    print(f"\n \"{text}\"")
    print(f"{'─'*60}")
    print(f"Full FT ->  {label_names[pred_full].upper():10s} ({probs_full[pred_full]*100:.1f}%)")
    print(f"LoRA    ->  {label_names[pred_lora].upper():10s} ({probs_lora[pred_lora]*100:.1f}%)")

    if pred_full == pred_lora:
        print(f":D Agree: {label_names[pred_full].upper()}")
    else:
        print(f":( Disagree! {label_names[pred_full].upper()} vs {label_names[pred_lora].upper()}")

    return {
        'text': text,
        'full_ft': label_names[pred_full],
        'full_conf': float(probs_full[pred_full]),
        'lora': label_names[pred_lora],
        'lora_conf': float(probs_lora[pred_lora]),
        'agree': pred_full == pred_lora
    }

# Test examples
examples = [
    "I'm so excited about this!",
    "I feel really sad and alone",
    "I love you so much",
    "This makes me so angry!",
    "Oh, great. Another day of happiness, just wonderful!"
]

print("\nTesting examples:")
for ex in examples:
    predict_emotion(ex)




Testing examples:

 "I'm so excited about this!"
────────────────────────────────────────────────────────────
Full FT ->  JOY        (99.8%)
LoRA    ->  JOY        (99.8%)
:D Agree: JOY

 "I feel really sad and alone"
────────────────────────────────────────────────────────────
Full FT ->  SADNESS    (99.9%)
LoRA    ->  SADNESS    (100.0%)
:D Agree: SADNESS

 "I love you so much"
────────────────────────────────────────────────────────────
Full FT ->  LOVE       (92.1%)
LoRA    ->  LOVE       (97.8%)
:D Agree: LOVE

 "This makes me so angry!"
────────────────────────────────────────────────────────────
Full FT ->  ANGER      (99.8%)
LoRA    ->  ANGER      (100.0%)
:D Agree: ANGER

 "Oh, great. Another day of happiness, just wonderful!"
────────────────────────────────────────────────────────────
Full FT ->  JOY        (99.8%)
LoRA    ->  JOY        (98.7%)
:D Agree: JOY


# Experimenting on textx that are longer

In [19]:
# Test examples
longexamples = [
    "I have been waiting for this moment for months, and now it is finally here! I cannot contain my excitement—everything just seems so perfect right now. I feel like I am on top of the world, and nothing could ruin my mood. The energy in the air is electrifying, and I cannot wait to see what happens next!",
    "Today, I had such a busy day at my internship. Work is piling up and I feel very anxious. Not to mention, there are so many upcoming examinations for school and I feel extremely unprepared. I have a vacation which I am really excited for and is looking forward to, however, it feels like the vacation is so far away. I am so close yet so far to the end of this semester. I feel like my health is declining. I will be really happy once I can catch a break.",
    "Last night, as I was lying in bed, I started thinking about the upcoming presentation at work. The anxiety hit me like a wave, and I couldn’t shake the feeling that something would go horribly wrong. I started imagining all the worst-case scenarios: forgetting my lines, technical difficulties, or worst of all, having people laugh at my ideas. The more I thought about it, the more nervous I became. My heart started racing, my palms got sweaty, and it felt like I couldn’t breathe properly. Even though I tried to calm myself down, the fear kept growing. It’s not the first time I’ve felt this way before a big event, but it never gets easier. I kept telling myself that everything would be fine, but the fear still wouldn’t let go.",
    "yesterday was one of those rare days where everything felt like they all fell into place. I woke up early and felt major gratitude for everything I have been blessed with. For the first time in a long while, I felt completely at peace.There was a lightness in the air, a sense of freedom that I haven’t felt in a long time. By the time I got home, I had this warm feeling in my chest, like everything was right in the world. I took in the world around me and it felt like nothing in this world can bring me down. I was present in the moment and took a deep breathe, taking time to savour and smell the roses and the fresh air around me. It felt like the sky was so clear, my skin was glowing, the weather was not too cold, not too humid"
]

print("\nTesting examples:")
for ex in longexamples:
    predict_emotion(ex)



Testing examples:

 "I have been waiting for this moment for months, and now it is finally here! I cannot contain my excitement—everything just seems so perfect right now. I feel like I am on top of the world, and nothing could ruin my mood. The energy in the air is electrifying, and I cannot wait to see what happens next!"
────────────────────────────────────────────────────────────
Full FT ->  JOY        (99.9%)
LoRA    ->  JOY        (99.4%)
:D Agree: JOY

 "Today, I had such a busy day at my internship. Work is piling up and I feel very anxious. Not to mention, there are so many upcoming examinations for school and I feel extremely unprepared. I have a vacation which I am really excited for and is looking forward to, however, it feels like the vacation is so far away. I am so close yet so far to the end of this semester. I feel like my health is declining. I will be really happy once I can catch a break."
────────────────────────────────────────────────────────────
Full FT ->  FEA

# Examples with multiple emotion keywords (eg. yesterday i was fustrated but today I am happy)

In [11]:
contradictingexamples = [
  "Yesterday, I was overjoyed about my promotion at work, but my excitement quickly turned to frustration when I found out that I had to handle more responsibility without any additional support from the team"
  , "When I opened the door yesterday, I was surprised to find a package I wasn’t expecting. But then, as I opened it, I started feeling anxious about what might be inside. what if it was something important I had missed?",
  "I felt a deep sadness yesterday when I realized how much I miss my family. Even though I love them and they love me, the distance has created an aching feeling that I can’t seem to shake."

  ]


print("\nTesting examples:")
for ex in contradictingexamples:
    predict_emotion(ex)




Testing examples:

 "Yesterday, I was overjoyed about my promotion at work, but my excitement quickly turned to frustration when I found out that I had to handle more responsibility without any additional support from the team"
────────────────────────────────────────────────────────────
Full FT → JOY        (99.6%)
LoRA    → JOY        (94.2%)
:D Agree: JOY

 "When I opened the door yesterday, I was surprised to find a package I wasn’t expecting. But then, as I opened it, I started feeling anxious about what might be inside. what if it was something important I had missed?"
────────────────────────────────────────────────────────────
Full FT → FEAR       (99.8%)
LoRA    → FEAR       (97.9%)
:D Agree: FEAR

 "I felt a deep sadness yesterday when I realized how much I miss my family. Even though I love them and they love me, the distance has created an aching feeling that I can’t seem to shake."
────────────────────────────────────────────────────────────
Full FT → SADNESS    (99.9%)
L

# Examples that have sacarstic tones

In [20]:
ironicexamples = [
  "lol, it's okay, I wasn't planning on sleeping anyways. Who needs sleep when there are deadlines to meet?",
  "I’m absolutely thrilled to handle this impossible task you just dropped on my lap",
  "Oh great! another teams message I didn’t ask for. And right when I was planning to leave the office. Just what I needed to brighten my day."

  ]


print("\nTesting examples:")
for ex in ironicexamples:
    predict_emotion(ex)


Testing examples:

 "lol, it's okay, I wasn't planning on sleeping anyways. Who needs sleep when there are deadlines to meet?"
────────────────────────────────────────────────────────────
Full FT ->  JOY        (62.7%)
LoRA    ->  JOY        (42.7%)
:D Agree: JOY

 "I’m absolutely thrilled to handle this impossible task you just dropped on my lap"
────────────────────────────────────────────────────────────
Full FT ->  JOY        (99.9%)
LoRA    ->  JOY        (99.6%)
:D Agree: JOY

 "Oh great! another teams message I didn’t ask for. And right when I was planning to leave the office. Just what I needed to brighten my day."
────────────────────────────────────────────────────────────
Full FT ->  JOY        (98.5%)
LoRA    ->  JOY        (97.5%)
:D Agree: JOY


# Mapping DSM-5 Symptoms to Emotions

In [23]:
dsm5examples = [
  "Depressed mood most of the day, nearly every day; markedly diminished interest or pleasure in all or almost all activities; significant weight loss or weight gain; insomnia or excessive sleeping; fatigue or loss of energy; feelings of worthlessness or excessive guilt; diminished ability to think or concentrate, or indecisiveness; thoughts of death or suicide, or suicide attempts.",
  "Excessive worry occurring more days than not for at least six months; difficulty controlling the worry; restlessness or feeling on edge; easily fatigued; difficulty concentrating or mind going blank; irritability; muscle tension; sleep disturbance.",
  "Restricting food intake leading to significantly low body weight (Anorexia); recurrent episodes of binge eating followed by inappropriate compensatory behaviors (Bulimia); extreme concern about body image and weight; distorted perception of weight or body size.",
  "Persistent pattern of inattention and/or hyperactivity-impulsivity that interferes with functioning or development; difficulty focusing on tasks, forgetfulness, impulsivity, restlessness, and a lack of organization."

  ]


print("\nTesting examples:")
for ex in dsm5examples:
    predict_emotion(ex)


Testing examples:

 "Depressed mood most of the day, nearly every day; markedly diminished interest or pleasure in all or almost all activities; significant weight loss or weight gain; insomnia or excessive sleeping; fatigue or loss of energy; feelings of worthlessness or excessive guilt; diminished ability to think or concentrate, or indecisiveness; thoughts of death or suicide, or suicide attempts."
────────────────────────────────────────────────────────────
Full FT ->  SADNESS    (99.9%)
LoRA    ->  SADNESS    (99.9%)
:D Agree: SADNESS

 "Excessive worry occurring more days than not for at least six months; difficulty controlling the worry; restlessness or feeling on edge; easily fatigued; difficulty concentrating or mind going blank; irritability; muscle tension; sleep disturbance."
────────────────────────────────────────────────────────────
Full FT ->  SADNESS    (92.9%)
LoRA    ->  FEAR       (84.2%)
:( Disagree! SADNESS vs FEAR

 "Restricting food intake leading to significan

# Interactive Section!
You can key in your own sentence in the text bar :D If you want to exit, just press ENter in the space bar

In [24]:
# Interactive input

print("Type sentences (Enter to stop):")


user_tests = []
while True:
    text = input("\n  Sentence: ").strip()
    if not text:
        break
    result = predict_emotion(text)


print("\nDemo complete!")

Type sentences (Enter to stop):

  Sentence: 

Demo complete!
